# SQLAlchemy Joins

## Setup

In [7]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import pandas as pd

In [2]:
engine = create_engine("sqlite:///../Resources/mammal_masses.sqlite", echo=False)

In [3]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['ea', 'na']

In [4]:
# Map Europe class
EA = Base.classes.ea

In [10]:
pd.DataFrame(inspect(engine).get_columns('ea'))

,name,type,nullable,default,autoincrement,primary_key
0,id,INTEGER,False,None,auto,1
1,record_id,INTEGER,True,None,auto,0
2,continent,TEXT,True,None,auto,0
3,status,TEXT,True,None,auto,0
4,sporder,TEXT,True,None,auto,0
5,family,TEXT,True,None,auto,0
6,genus,TEXT,True,None,auto,0
7,species,TEXT,True,None,auto,0
8,log_mass_g,FLOAT,True,None,auto,0
9,comb_mass_g,FLOAT,True,None,auto,0


In [5]:
# Map North American class
NA = Base.classes.na

In [6]:
# create a session
session = Session(engine)

## Filtering Review

Filters are the "WHERE" clause for your select statement. 

In [11]:
# filter North American mammals whose genus is "Antilocapra"
# query, loop over and print out animals.
mammals = session.query(NA).filter(NA.genus == 'Antilocapra').all()
for mammal in mammals:
    print("Family: {0}, Genus: {1}".format(mammal.family, mammal.genus))

Family: Antilocapridae, Genus: Antilocapra


## Joins

A SQL join combines columns from one or more tables in a relational database. 

It creates a set that can be saved as a table or used as it is. 

A JOIN is a means for combining columns from one (self-table) or more tables by using values common to each.

In [12]:
inspector = inspect(engine)
inspector.get_table_names()

['ea', 'na']

In [13]:
# Get a list of column names and types
columns = inspector.get_columns('ea')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
record_id INTEGER
continent TEXT
status TEXT
sporder TEXT
family TEXT
genus TEXT
species TEXT
log_mass_g FLOAT
comb_mass_g FLOAT
reference TEXT


In [10]:
session.query(EA.sporder, NA.sporder).limit(100).all()

[('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'A

In [11]:
same_sporder = session.query(EA, NA).filter(EA.sporder == NA.sporder).limit(10).all()


for record in same_sporder:
    (ea, na) = record
    print(ea.sporder)
    print(na.sporder)

Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla
Artiodactyla


In [14]:
print(session.query(EA, NA).filter(EA.sporder == NA.sporder).limit(10))

SELECT ea.id AS ea_id, ea.record_id AS ea_record_id, ea.continent AS ea_continent, ea.status AS ea_status, ea.sporder AS ea_sporder, ea.family AS ea_family, ea.genus AS ea_genus, ea.species AS ea_species, ea.log_mass_g AS ea_log_mass_g, ea.comb_mass_g AS ea_comb_mass_g, ea.reference AS ea_reference, na.id AS na_id, na.record_id AS na_record_id, na.continent AS na_continent, na.status AS na_status, na.sporder AS na_sporder, na.family AS na_family, na.genus AS na_genus, na.species AS na_species, na.log_mass_g AS na_log_mass_g, na.comb_mass_g AS na_comb_mass_g, na.reference AS na_reference 
FROM ea, na 
WHERE ea.sporder = na.sporder
 LIMIT ? OFFSET ?


In [12]:
# Return all animals from EA and NA belonging to the same sporder.
# This JOINs the data in the two tables together into a single dataset (here in the form of a tuple).
# Note: We are going to limit the results to 10 for printing
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_sporder = session.query(*sel).filter(EA.sporder == NA.sporder).limit(10).all()

for record in same_sporder:
    (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = record
    print(
        f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
        f"belongs to the same sporder as the North American animal '{na_fam} {na_gen} {na_spec}'.")

The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Antilocapra americana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Capromeryx mexicana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Capromeryx minor'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Stockoceros conklingi'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Stockoceros onusrosagris'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Tetrameryx shuleri'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Bovidae Bison bison'.
Th